In [ ]:
%pip install scikit-learn==1.7.1

In [3]:
import pandas as pd
import joblib
import sys

In [4]:
try:
    rf_model = joblib.load('random_forest_model.pkl')
except FileNotFoundError:
    print("Error: 'random_forest_model.pkl' not found in the current directory.")
    sys.exit(1)
except Exception as e:
    print(f"Error loading model: {str(e)}")
    sys.exit(1)

In [ ]:
# Validation function for user input
def validate_input(prompt, name, cast_type, min_val=None, max_val=None, allowed=None):
    while True:
        try:
            print(prompt)
            value = cast_type(input(f"Enter {name}: "))
            if allowed is not None and value not in allowed:
                print(f"Error: {name} must be one of {allowed}. Try again.")
                continue
            if min_val is not None and max_val is not None and not (min_val <= value <= max_val):
                print(f"Error: {name} must be between {min_val} and {max_val}. Try again.")
                continue
            return value
        except ValueError:
            print(f"Error: Please enter a valid {cast_type.__name__} for {name}.")
        except KeyboardInterrupt:
            print("\nExiting program.")
            sys.exit(0)

# Welcome message
print("\n=== Heart Failure Risk Predictor ===")
print("This tool uses a pre-trained Random Forest model to predict heart failure risk.")
print("Enter patient details as prompted. All inputs are validated with realistic medical ranges.")

# Feature order must match the training data
feature_order = [
    'age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction',
    'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking'
]

while True:
    # Collect patient input
    patient_input = {}
    patient_input['age'] = validate_input(
        "Age (years, 18–120):", "Age", int, min_val=18, max_val=120
    )
    patient_input['anaemia'] = validate_input(
        "Anaemia (0 = No, 1 = Yes):", "Anaemia", int, allowed={0, 1}
    )
    patient_input['creatinine_phosphokinase'] = validate_input(
        "Creatinine Phosphokinase (µg/L, 10–10000):", "Creatinine Phosphokinase", int, min_val=10, max_val=10000
    )
    patient_input['diabetes'] = validate_input(
        "Diabetes (0 = No, 1 = Yes):", "Diabetes", int, allowed={0, 1}
    )
    patient_input['ejection_fraction'] = validate_input(
        "Ejection Fraction (%, 10–80):", "Ejection Fraction", int, min_val=10, max_val=80
    )
    patient_input['high_blood_pressure'] = validate_input(
        "High Blood Pressure (0 = No, 1 = Yes):", "High Blood Pressure", int, allowed={0, 1}
    )
    patient_input['platelets'] = validate_input(
        "Platelets (kiloplatelets/µL, 50000–1000000):", "Platelets", float, min_val=50000, max_val=1000000
    )
    patient_input['serum_creatinine'] = validate_input(
        "Serum Creatinine (mg/dL, 0.1–15):", "Serum Creatinine", float, min_val=0.1, max_val=15
    )
    patient_input['serum_sodium'] = validate_input(
        "Serum Sodium (mEq/L, 110–160):", "Serum Sodium", int, min_val=110, max_val=160
    )
    patient_input['sex'] = validate_input(
        "Sex (0 = Female, 1 = Male):", "Sex", int, allowed={0, 1}
    )
    patient_input['smoking'] = validate_input(
        "Smoking (0 = No, 1 = Yes):", "Smoking", int, allowed={0, 1}
    )

    # Create patient DataFrame
    patient_df = pd.DataFrame([patient_input])[feature_order]

    # Make prediction
    try:
        prediction = rf_model.predict(patient_df)[0]
        probability = rf_model.predict_proba(patient_df)[0, 1]
    except Exception as e:
        print(f"Error during prediction: {str(e)}")
        sys.exit(1)

    # Interpret probability for risk level
    risk_level = "High Risk" if probability >= 0.7 else "Moderate Risk" if probability >= 0.3 else "Low Risk"

    # Output results
    status = "Died" if prediction == 1 else "Survived"
    print("\n=== Heart Failure Risk Prediction ===")
    print(f"Predicted Outcome: {status}")
    print(f"Probability of Death: {probability:.2f} ({risk_level})")

    # Ask to run another prediction
    try:
        again = input("Would you like to predict for another patient? (yes/no): ").strip().lower()
        if again != 'yes':
            print("Exiting program.")
            break
    except KeyboardInterrupt:
        print("\nExiting program.")
        break